<a href="https://colab.research.google.com/github/prakul/MongoDB-AI-Resources/blob/main/Retrievers_mongodb_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment

In [ ]:
!pip install -qU langchain-mongodb langchain-openai python-dotenv pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.8 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
MONGO_URI = os.environ["MONGO_URI"]

# Setup



In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient

# use MongoDB movies database

DB_NAME = "sample_mflix"
COLLECTION_NAME = "embedded_movies"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "test"
#TIMEOUT = 60.0

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
MONGODB_COLLECTION = db[COLLECTION_NAME]

In [ ]:
for p in MONGODB_COLLECTION.list_search_indexes():
    print(p)

In [ ]:
embedding_model= OpenAIEmbeddings(disallowed_special=())
DIMENSIONS = 1536

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
vectorstore = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding= embedding_model,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    text_key="plot",
    embedding_key="plot_embedding")


In [ ]:
vectorstore.create_vector_search_index(DIMENSIONS)


In [ ]:
for p in MONGODB_COLLECTION.list_search_indexes():
    print(p)

{'id': '672ea9979b770656c39b2719', 'name': 'test', 'type': 'vectorSearch', 'status': 'READY', 'queryable': True, 'latestDefinitionVersion': {'version': 0, 'createdAt': datetime.datetime(2024, 11, 9, 0, 15, 20, 45000)}, 'latestDefinition': {'fields': [{'numDimensions': 1536, 'path': 'plot_embedding', 'similarity': 'cosine', 'type': 'vector'}]}, 'statusDetail': [{'hostname': 'atlas-3304g1-shard-00-00', 'status': 'READY', 'queryable': True, 'mainIndex': {'status': 'READY', 'queryable': True, 'definitionVersion': {'version': 0, 'createdAt': datetime.datetime(2024, 11, 9, 0, 15, 20)}, 'definition': {'fields': [{'type': 'vector', 'path': 'plot_embedding', 'numDimensions': 1536, 'similarity': 'cosine'}]}}}, {'hostname': 'atlas-3304g1-shard-00-02', 'status': 'READY', 'queryable': True, 'mainIndex': {'status': 'READY', 'queryable': True, 'definitionVersion': {'version': 0, 'createdAt': datetime.datetime(2024, 11, 9, 0, 15, 20)}, 'definition': {'fields': [{'type': 'vector', 'path': 'plot_embeddi

# Vector Retriever

## Simple query

In [ ]:
query = "Comedy movies with imaginary characters"

In [ ]:
docs = vectorstore.similarity_search(query)
for doc in docs:
    print("# " + doc.metadata["title"])
    print(doc.page_content[:100])
    print(doc.metadata["year"])


# Last Action Hero
With the help of a magic ticket, a young film fan is transported into the fictional world of his fav
1993
# Showtime
A spoof of buddy cop movies where two very different cops are forced to team up on a new reality bas
2002
# The Gods Must Be Crazy
A comic allegory about a traveling Bushman who encounters modern civilization and its stranger aspec
1980
# King Size
A comedy. The story follows a young scientist in the contemporary world, who actually came from the 
1988


## Complex query

In [ ]:
query = "Comedy movies with imaginary characters released between 1980 & 1990"

In [ ]:
docs = vectorstore.similarity_search(query)
for doc in docs:
    print("# " + doc.metadata["title"])
    print(doc.page_content[:100])
    print(doc.metadata["year"])

# The Purple Rose of Cairo
In 1930s New Jersey, a movie character walks off the screen and into the real world.
1985
# Pleasantville
Two 1990s teenagers find themselves in a 1950s sitcom where their influence begins to profoundly cha
1998
# Last Action Hero
With the help of a magic ticket, a young film fan is transported into the fictional world of his fav
1993
# Fat Albert
Fat Albert and the Cosby Kids come to life and step out of their animated, inner-city Philadelphia w
2004


# Metadata Pre-filtering

In [ ]:
# Update the vector search index to filter on year
vectorstore.create_vector_search_index(1536, filters=["year"], update=True)

## Query Re-writer

In [ ]:
query = "Comedy movies with imaginary characters released between 1980 & 1990"

In [ ]:
### Question Re-writer
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

def rewrite_query(query):
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
    re_write_prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
                """You a question re-writer that converts an input question to a better version that is optimized \n
                for vector search retrieval. Look at the input and if it contains date range return a JSON with original query , start year, end year.
                Otherwise return a JSON with original query.
                """

            ),
            (
                "human",
                    "Here is the initial question: \n\n {question} \n Formulate an improved question.",
            ),
        ]
    )

    question_rewriter = re_write_prompt | llm | StrOutputParser()
    return question_rewriter.invoke({"question": query})


In [ ]:
rewritten_query = rewrite_query(query)
print(rewritten_query)

{
    "original_query": "Comedy movies with imaginary characters released between 1980 & 1990",
    "start_year": 1980,
    "end_year": 1990
}


In [ ]:
import json

def get_mql_filter(rewritten_query):
    q = json.loads(rewritten_query)
    return {
        '$and': [
                 {
            'year': {
              '$lt': q['end_year']
            },
                 },
                 {

            'year': {
              '$gt': q['start_year']
            }

          }
        ]
      }


In [ ]:
mql = get_mql_filter(rewritten_query)
print(mql)
docs = vectorstore.similarity_search(query, pre_filter=mql)
for doc in docs:
    print("# " + doc.metadata["title"])
    print(doc.page_content[:100])
    print(doc.metadata["year"])

{'$and': [{'year': {'$lt': 1990}}, {'year': {'$gt': 1980}}]}
# The Purple Rose of Cairo
In 1930s New Jersey, a movie character walks off the screen and into the real world.
1985
# Hercules
The story of the Greek mythological figure, updated in this 80's version.
1983
# King Size
A comedy. The story follows a young scientist in the contemporary world, who actually came from the 
1988
# Armour of God
Jackie Chan and his bumbling sidekick are sent on a quest through Europe to find a mysterious treasu
1986


# RAG

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = vectorstore.as_retriever()
# Prompt
prompt = hub.pull("rlm/rag-prompt")

# Chain
rag_chain = (
    {"context": retriever ,  "question": RunnablePassthrough()}
    | prompt
    | llm
)


In [ ]:
query = "Suggest me a movie that can lift my spirits. I like fiction."

In [ ]:
rag_result = rag_chain.invoke(query)
print(rag_result)

content='I suggest you watch "Last Action Hero." It is a fiction movie that can lift your spirits with its action-packed and comedic storyline. The film features Arnold Schwarzenegger as a larger-than-life action hero.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 2552, 'total_tokens': 2595, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-17826cdb-5933-446c-bc97-08d39eea848e-0' usage_metadata={'input_tokens': 2552, 'output_tokens': 43, 'total_tokens': 2595, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


In [ ]:
rag_result = rag_chain.invoke("I dont like this one. Suggest me another")
print(rag_result)

content="I don't know." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 2546, 'total_tokens': 2551, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4737d1ad-f8c7-4f87-8441-8de4a3f1df1d-0' usage_metadata={'input_tokens': 2546, 'output_tokens': 5, 'total_tokens': 2551, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


# Convo History

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory



In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI()

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: MongoDBChatMessageHistory(MONGO_URI, session_id, database_name=DB_NAME, collection_name="history"),
    input_messages_key="question",
    history_messages_key="history",
)



In [ ]:
# This is where we configure the session id
config = {"configurable": {"session_id": "<SESSION_ID>"}}


In [ ]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config).content



'Hello Bob! How can I assist you today?'

In [ ]:
chain_with_history.invoke({"question": "Whats my name"}, config=config).content

'Your name is Bob.'